In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [3]:
#read data from previous data science project saved csv file to dataframe using project library
new_file = project.get_file("torontolatlong.csv")
my_df = pd.read_csv(new_file)
my_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Rouge,43.806686,-79.194353
1,M1B,Scarborough,Malvern,43.806686,-79.194353
2,M1C,Scarborough,Highland Creek,43.784535,-79.160497
3,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
4,M1C,Scarborough,Port Union,43.784535,-79.160497
5,M1E,Scarborough,Guildwood,43.763573,-79.188711
6,M1E,Scarborough,Morningside,43.763573,-79.188711
7,M1E,Scarborough,West Hill,43.763573,-79.188711
8,M1G,Scarborough,Woburn,43.770992,-79.216917
9,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [4]:
# Read Toronto neighbourhoods census data, after removing irrelevant categories, from csv file downloaded from public website
# Categories kept were population in 2011 and 2016, change in population from 2011 to 2016, Ethnic origin - South Asian includes Indians
new2_file = project.get_file("2016_neighbourhood_profiles_edited.csv")
p_df = pd.read_csv(new2_file, encoding = "ISO-8859-1")
p_df

,Category,Characteristic,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,Dufferin Grove,East End-Danforth,Edenbridge-Humber Valley,Eglinton East,Elms-Old Rexdale,Englemount-Lawrence,Eringate-Centennial-West Deane,Etobicoke West Mall,Flemingdon Park,Forest Hill North,Forest Hill South,Glenfield-Jane Heights,Greenwood-Coxwell,Guildwood,Henry Farm,High Park North,High Park-Swansea,Highland Creek,Hillcrest Village,Humber Heights-Westmount,Humber Summit,Humbermede,Humewood-Cedarvale,Ionview,Islington-City Centre West,Junction Area,Keelesdale-Eglinton West,Kennedy Park,Kensington-Chinatown,Kingsview Village-The Westway,Kingsway South,Lambton Baby Point,L'Amoreaux,Lansing-Westgate,Lawrence Park North,Lawrence Park South,Leaside-Bennington,Little Portugal,Long Branch,Malvern,Maple Leaf,Markland Wood,Milliken,Mimico (includes Humber Bay Shores),Morningside,Moss Park,Mount Dennis,Mount Olive-Silverstone-Jamestown,Mount Pleasant East,Mount Pleasant West,New Toronto,Newtonbrook East,Newtonbrook West,Niagara,North Riverdale,North St. James Town,Oakridge,Oakwood Village,O'Connor-Parkview,Old East York,Palmerston-Little Italy,Parkwoods-Donalda,Pelmo Park-Humberlea,Playter Estates-Danforth,Pleasant View,Princess-Rosethorn,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,Population,"Population, 2016","29,113","23,757","12,054","30,526","27,695","15,873","25,797","21,396","13,154","23,236","6,577","29,960","22,291","21,737","7,727","14,257","9,266","11,499","17,757","11,669","9,955","10,968","13,362","31,340","26,984","16,472","15,935","14,133","9,666","17,180","27,051","25,003","36,625","35,052","11,785","21,381","15,535","22,776","9,456","22,372","18,588","11,848","21,933","12,806","10,732","30,491","14,417","9,917","15,723","22,162","23,925","12,494","16,934","10,948","12,416","15,545","14,365","13,641","43,965","14,366","11,058","17,123","17,945","22,000","9,271","7,985","43,993","16,164","14,607","15,179","16,828","15,559","10,084","43,794","10,111","10,554","26,572","33,964","17,455","20,506","13,593","32,954","16,775","29,658","11,463","16,097","23,831","31,180","11,916","18,615","13,845","21,210","18,675","9,233","13,826","34,805","10,722","7,804","15,818","11,051","10,803","10,529","22,246","14,974","20,923","46,496","10,070","9,941","16,724","21,849","27,876","17,812","24,623","25,051","27,446","15,683","21,567","10,360","21,108","16,556","7,607","17,510","65,913","27,392","33,312","26,274","17,992","11,098","27,917","50,434","16,936","22,156","53,485","12,541","7,865","14,349","11,817","12,528","27,593","14,804"
1,Population,"Population, 2011","30,279","21,988","11,904","29,177","26,918","15,434","19,348","17,671","13,530","23,185","6,488","27,876","21,856","22,057","7,763","14,302","8,713","11,563","17,787","12,053","9,851","10,487","13,0

In [5]:
# Clean up and transfom census data for subsequent use
new_df = p_df.transpose()
new_df

,0,1,2,3,4
Category,Population,Population,Population,Ethnic origin,Income
Characteristic,"Population, 2016","Population, 2011",Population Change 2011-2016,South Asian origins,"70,000 and over"
Agincourt North,"29,113","30,279",-3.90%,"5,260",2120
Agincourt South-Malvern West,"23,757","21,988",8.00%,"4,170",2005
Alderwood,"12,054","11,904",1.30%,575,2140
Annex,"30,526","29,177",4.60%,"1,320",8375
Banbury-Don Mills,"27,695","26,918",2.90%,"2,940",6670
Bathurst Manor,"15,873","15,434",2.80%,565,2400
Bay Street Corridor,"25,797","19,348",33.30%,"2,515",5465
Bayview Village,"21,396","17,671",21.10%,"1,410",4165


In [6]:
# Clean up and transfom census data for subsequent use
new_df.drop(["Category"], axis=0, inplace = True)
new_df

,0,1,2,3,4
Characteristic,"Population, 2016","Population, 2011",Population Change 2011-2016,South Asian origins,"70,000 and over"
Agincourt North,"29,113","30,279",-3.90%,"5,260",2120
Agincourt South-Malvern West,"23,757","21,988",8.00%,"4,170",2005
Alderwood,"12,054","11,904",1.30%,575,2140
Annex,"30,526","29,177",4.60%,"1,320",8375
Banbury-Don Mills,"27,695","26,918",2.90%,"2,940",6670
Bathurst Manor,"15,873","15,434",2.80%,565,2400
Bay Street Corridor,"25,797","19,348",33.30%,"2,515",5465
Bayview Village,"21,396","17,671",21.10%,"1,410",4165
Bayview Woods-Steeles,"13,154","13,530",-2.80%,805,2310


In [7]:
# Clean up and transfom census data for subsequent use
new_df.columns = new_df.iloc[0]
new_df.drop(["Characteristic"], axis=0, inplace = True)
new_df

Characteristic,"Population, 2016","Population, 2011",Population Change 2011-2016,South Asian origins,"70,000 and over"
Agincourt North,"29,113","30,279",-3.90%,"5,260",2120
Agincourt South-Malvern West,"23,757","21,988",8.00%,"4,170",2005
Alderwood,"12,054","11,904",1.30%,575,2140
Annex,"30,526","29,177",4.60%,"1,320",8375
Banbury-Don Mills,"27,695","26,918",2.90%,"2,940",6670
Bathurst Manor,"15,873","15,434",2.80%,565,2400
Bay Street Corridor,"25,797","19,348",33.30%,"2,515",5465
Bayview Village,"21,396","17,671",21.10%,"1,410",4165
Bayview Woods-Steeles,"13,154","13,530",-2.80%,805,2310
Bedford Park-Nortown,"23,236","23,185",0.20%,535,7055


In [8]:
# Clean up and transfom census data for subsequent use
new_df.reset_index(inplace=True)
o_df = new_df
o_df

Characteristic,index,"Population, 2016","Population, 2011",Population Change 2011-2016,South Asian origins,"70,000 and over"
0,Agincourt North,"29,113","30,279",-3.90%,"5,260",2120
1,Agincourt South-Malvern West,"23,757","21,988",8.00%,"4,170",2005
2,Alderwood,"12,054","11,904",1.30%,575,2140
3,Annex,"30,526","29,177",4.60%,"1,320",8375
4,Banbury-Don Mills,"27,695","26,918",2.90%,"2,940",6670
5,Bathurst Manor,"15,873","15,434",2.80%,565,2400
6,Bay Street Corridor,"25,797","19,348",33.30%,"2,515",5465
7,Bayview Village,"21,396","17,671",21.10%,"1,410",4165
8,Bayview Woods-Steeles,"13,154","13,530",-2.80%,805,2310
9,Bedford Park-Nortown,"23,236","23,185",0.20%,535,7055


In [9]:
# Clean up and transfom census data for subsequent use
o_df.rename(columns={"index":"Neighbourhood"}, inplace=True)
#project.save_data("neighborhood.csv", o_df.to_csv()) # save file for reuse, commented out to prevent overwrite

In [10]:
# Read census data from saved file
new_file = project.get_file("neighborhood.csv")
r_df = pd.read_csv(new_file)
r_df

,Unnamed: 0,Neighbourhood,"Population, 2016","Population, 2011",Population Change 2011-2016,South Asian origins,"70,000 and over"
0,0,Agincourt North,29113,30279,-0.04,5260,2120
1,1,Agincourt,23757,21988,0.08,4170,2005
2,2,Alderwood,12054,11904,0.01,575,2140
3,3,The Annex,30526,29177,0.05,1320,8375
4,4,Don Mills North,27695,26918,0.03,2940,6670
5,5,Bathurst Manor,15873,15434,0.03,565,2400
6,6,Central Bay Street,25797,19348,0.33,2515,5465
7,7,Bayview Village,21396,17671,0.21,1410,4165
8,8,South Steeles,13154,13530,-0.03,805,2310
9,9,Bedford Park,23236,23185,0.00,535,7055


In [11]:
# Merge census data and neighbourhood Geo data 
df_merged = pd.merge(my_df, r_df, on='Neighbourhood')
df_merged
#project.save_data("merged.csv", df_merged.to_csv()) # save merged data for reuse, commented out to prevent overwrite

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Unnamed: 0,"Population, 2016","Population, 2011",Population Change 2011-2016,South Asian origins,"70,000 and over"
0,M1B,Scarborough,Rouge,43.806686,-79.194353,105,46496,45912,0.01,20265,5585
1,M1B,Scarborough,Malvern,43.806686,-79.194353,73,43794,45086,-0.03,18085,2265
2,M1C,Scarborough,Highland Creek,43.784535,-79.160497,51,12494,13097,-0.05,4685,1790
3,M1E,Scarborough,Guildwood,43.763573,-79.188711,47,9917,9816,0.01,970,2155
4,M1E,Scarborough,Morningside,43.763573,-79.188711,78,17455,17587,-0.01,5370,1300
5,M1E,Scarborough,West Hill,43.763573,-79.188711,123,27392,26547,0.03,5420,2165
6,M1G,Scarborough,Woburn,43.770992,-79.216917,132,53485,53350,0.00,21505,3655
7,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,22,13362,13093,0.02,2525,3040
8,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,108,16724,16609,0.01,5535,1135
9,M1K,Scarborough,Ionview,43.727929,-79.262029,57,13641,13091,0.04,3845,825


In [12]:
# Explore merged data by Neighbourhoods with Highest earning households (over 70,000 $ per annum)
df_merged = df_merged.sort_values(by ='70,000 and over', ascending=False)
df_merged.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Unnamed: 0,"Population, 2016","Population, 2011",Population Change 2011-2016,South Asian origins,"70,000 and over"
57,M5V,Downtown Toronto,South Niagara,43.628947,-79.394420,87,31180,21274,0.47,2420,10140
26,M2M,North York,Willowdale,43.789053,-79.408493,129,50434,45041,0.12,3460,9015
83,M9B,Etobicoke,Islington,43.650943,-79.554724,58,43965,38084,0.15,3595,8435
54,M5R,Central Toronto,The Annex,43.672710,-79.405678,3,30526,29177,0.05,1320,8375
44,M4T,Central Toronto,Moore Park,43.689574,-79.383160,104,20923,20631,0.01,685,7905
75,M8V,Etobicoke,Mimico South,43.605647,-79.501321,77,33964,26541,0.28,1940,7630
49,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,23,31340,28349,0.11,2730,7310
52,M5M,North York,Bedford Park,43.733283,-79.419750,9,23236,23185,0.00,535,7055
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,116,21567,21130,0.02,795,6745
29,M3B,North York,Don Mills North,43.745906,-79.352188,4,27695,26918,0.03,2940,6670


In [13]:
 #Explore merged data by Neighbourhoods with Highest population in 2016
df_merged = df_merged.sort_values(by ='Population, 2016', ascending=False)
df_merged.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Unnamed: 0,"Population, 2016","Population, 2011",Population Change 2011-2016,South Asian origins,"70,000 and over"
6,M1G,Scarborough,Woburn,43.770992,-79.216917,132,53485,53350,0.00,21505,3655
26,M2M,North York,Willowdale,43.789053,-79.408493,129,50434,45041,0.12,3460,9015
0,M1B,Scarborough,Rouge,43.806686,-79.194353,105,46496,45912,0.01,20265,5585
20,M1V,Scarborough,L'Amoreaux East,43.815252,-79.284577,66,43993,44919,-0.02,7265,3465
83,M9B,Etobicoke,Islington,43.650943,-79.554724,58,43965,38084,0.15,3595,8435
1,M1B,Scarborough,Malvern,43.806686,-79.194353,73,43794,45086,-0.03,18085,2265
61,M6H,West Toronto,Dovercourt Village,43.669005,-79.442259,32,36625,34631,0.06,2390,4995
65,M6L,North York,Downsview,43.713756,-79.490074,33,35052,34659,0.01,1685,2670
28,M3A,North York,Parkwoods,43.753259,-79.329656,95,34805,34617,0.01,3685,4575
75,M8V,Etobicoke,Mimico South,43.605647,-79.501321,77,33964,26541,0.28,1940,7630


In [14]:
#  Explore merged data by Neighbourhoods with Highest population change between 2011 and 2016 ie fastest growing
df_merged = df_merged.sort_values(by ='Population Change 2011-2016', ascending=False)
df_merged.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Unnamed: 0,"Population, 2016","Population, 2011",Population Change 2011-2016,South Asian origins,"70,000 and over"
57,M5V,Downtown Toronto,South Niagara,43.628947,-79.394420,87,31180,21274,0.47,2420,10140
23,M2J,North York,Henry Farm,43.778517,-79.346556,48,15723,11333,0.39,3295,1815
51,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,6,25797,19348,0.33,2515,5465
63,M6J,West Toronto,Little Portugal,43.647927,-79.419750,71,15559,12050,0.29,480,2675
75,M8V,Etobicoke,Mimico South,43.605647,-79.501321,77,33964,26541,0.28,1940,7630
24,M2K,North York,Bayview Village,43.786947,-79.385975,7,21396,17671,0.21,1410,4165
83,M9B,Etobicoke,Islington,43.650943,-79.554724,58,43965,38084,0.15,3595,8435
27,M2R,North York,Willowdale West,43.782736,-79.442259,130,16936,15004,0.13,935,3120
26,M2M,North York,Willowdale,43.789053,-79.408493,129,50434,45041,0.12,3460,9015
49,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,23,31340,28349,0.11,2730,7310


In [15]:
#  Explore merged data by Neighbourhoods with Highest number of people of indian origin
df_merged = df_merged.sort_values(by =' South Asian origins', ascending=False)
df_merged.head(10)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Unnamed: 0,"Population, 2016","Population, 2011",Population Change 2011-2016,South Asian origins,"70,000 and over"
6,M1G,Scarborough,Woburn,43.770992,-79.216917,132,53485,53350,0.00,21505,3655
0,M1B,Scarborough,Rouge,43.806686,-79.194353,105,46496,45912,0.01,20265,5585
1,M1B,Scarborough,Malvern,43.806686,-79.194353,73,43794,45086,-0.03,18085,2265
80,M8Y,Etobicoke,Humber Bay,43.636258,-79.498509,124,33312,34100,-0.02,14275,2225
93,M9V,Etobicoke,Mount Olive,43.739416,-79.588437,81,32954,32788,0.01,11395,1145
39,M4H,East York,Thorncliffe Park,43.705369,-79.349372,118,21108,19225,0.10,9545,1195
30,M3C,North York,Flemingdon Park,43.725900,-79.340923,42,21933,22168,-0.01,7460,1190
15,M1P,Scarborough,Dorset Park,43.757410,-79.273304,31,25003,24363,0.03,7380,1635
20,M1V,Scarborough,L'Amoreaux East,43.815252,-79.284577,66,43993,44919,-0.02,7265,3465
11,M1L,Scarborough,Clairlea,43.711112,-79.284577,24,26984,24770,0.09,6060,2805


In [16]:
# visualise neighbourhoods on map
#Use geopy library to get the latitude and longitude values of Toronto.
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [17]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Borough'], df_merged['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


In [18]:
# The code was removed by Watson Studio for sharing.

In [19]:
# Let's explore the first neighborhood in our dataframe. Get the neighborhood's name
df_merged.loc[0, 'Neighbourhood']

'Rouge'

In [20]:
# Get neighborhood's latitude and longitude
neighbourhood_latitude = df_merged.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = df_merged.loc[0, 'Longitude'] # neighbourhood longitude value
#categoryId = '4bf58dd8d48988d10f941735'

In [21]:
# Now, let's get the top 100 venues that are in Rouge within a radius of 2000 meters
LIMIT = 100
radius = 2000
#url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighbourhood_latitude, neighbourhood_longitude, VERSION, radius, LIMIT)
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighbourhood_latitude, neighbourhood_longitude, VERSION, radius, LIMIT)

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d2c428a6e4650002c5dfc6c'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4cd854fd3ec4b1f71900be3f-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/zoo_',
          'suffix': '.png'},
         'id': '58daa1558bbb0b01f18ec1fd',
         'name': 'Zoo Exhibit',
         'pluralName': 'Zoo Exhibits',
         'primary': True,
         'shortName': 'Zoo Exhibit'}],
       'id': '4cd854fd3ec4b1f71900be3f',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 1509,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.81772505914066,
          'lng': -79.18343284457424}],
        'lat': 43.81772505914

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,African Rainforest Pavilion,Zoo Exhibit,43.817725,-79.183433
1,Toronto Pan Am Sports Centre,Athletics & Sports,43.790623,-79.193869
2,Canadiana exhibit,Zoo Exhibit,43.817962,-79.193374
3,penguin exhibit,Zoo Exhibit,43.819435,-79.185959
4,Toronto Zoo,Zoo,43.820582,-79.181551


In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=4000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
#  code to run the above function on each neighborhood and create a new dataframe called toronto_venues
toronto_venues = getNearbyVenues(names=df_merged['Neighbourhood'],
                                   latitudes=df_merged['Latitude'],
                                   longitudes=df_merged['Longitude']
                                  )

Woburn
Rouge
Malvern
Humber Bay
Mount Olive
Thorncliffe Park
Flemingdon Park
Dorset Park
L'Amoreaux East
Clairlea
Scarborough Village
West Hill
Morningside
Agincourt North
Oakridge
Sullivan
Highland Creek
York University
Kennedy Park
Wexford
Agincourt
Ionview
St. James Town
St. James Town
Parkwoods
Kingsview Village
Islington
Humber Summit
Willowdale
Humbergate
Henry Farm
Milliken
Victoria Village
Cliffcrest
Don Mills North
Regent Park
Beaumond Heights
Church and Wellesley
Scarborough Village
Central Bay Street
Martin Grove
South Niagara
Dovercourt Village
Mimico South
Parkview Hill
Parkdale
Downsview
South Steeles
Birch Cliff
Hillcrest Village


In [ ]:
# check how many venues were returned for each neighborhood, most are at limit as we used a larger radius as not to miss anything
toronto_venues.groupby('Neighborhood').count()

In [ ]:
# Let's find out the unique categories that can be curated from all the returned venues
toronto_venues['Venue Category'].unique()
#project.save_data("venues.csv", toronto_venues.to_csv()), save for reuse

In [ ]:
# create new dataframe to see only indian restaurants in all neighbourhoods
ind_df = toronto_venues[toronto_venues['Venue Category']=="Indian Restaurant"]
ind_df

In [ ]:
# visualise distribution of Indian restaurants on map
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, venue, neighborhood in zip(ind_df['Venue Latitude'], ind_df['Venue Longitude'], ind_df['Venue'], ind_df['Neighborhood']):
    label = '{}, {}'.format(venue,neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)  
    
    
map_toronto

In [ ]:
# count number of indian restaurants in the neighborhoods that have them
gk = ind_df.groupby('Neighborhood')
gk.size()

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

In [ ]:
# Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
# write a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
# Run k-means to cluster the neighborhood into 5 clusters
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_merged

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

In [ ]:
# visualize the resulting clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
# examine each cluster 0-4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(2, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(2, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(2, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(2, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(2, toronto_merged.shape[1]))]]

On examinining the clusters we see that Cluster number 4 is of our interest as it has the highest number of people of South Asian origin and also Indian restaurants are prominent among the most common venues.
Combining data from our exploratory analysis on merged dataframe and Clustering it is clear that the following neighborhoods are ideal to set up a new Indian Restaurant - No Indian rstaurant, High population with high number of people of South Asian Origin.
 - Scarborough - Malvern
 - Scarborough - Rouge
 - Scarborough - Highland Creek
 - North York - York University
 - Etobicoke - Kingsview Village
 - York - Weston
 - Etobicoke - Westmount